# Issues with CAFE-f6

While spinning up this project, I discovered a number of issues with the CAFE-f6 forecasts that must be contended with when assessing the hindcasts.

These issues are breifly described/demonstrated in this notebook.

## Application of forcing

Some of the forcing fields in the CAFE-f6 forecasts are applied in a way that is different that the CMIP DCPP recomendations. Principally, a number of the forcing fields switch from time-varying to fixed based on the *initialisation date* of the forecasts. **This means that the same calendar year can experience different forcing, depending on its lead time.** For example (see below), The lead 2 year forecast of 2002 (initialised 2000) will be forced with 2002 volcanic forcing, but the lead 1 forecast of 2002 (initialised 2001) will be forced with 2008 forcing. This could make the forecasts difficult to interpret and baseline.

The relevant lines that specify the forcing in the CAFE-f6 model namelist (see https://github.com/csiro-dcfp/cm-forecasts/blob/master/ref/MODEL/input.in) are:

```
 &aerosol_nml
    use_aerosol_timeseries = .false.
    aerosol_dataset_entry =  INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
                             INPUT_AEROSOL_TIME,
    family_names = "small_dust", "large_dust", "sulfate", "aerosol"
    in_family1 = F,F,F,F,T,T,T,T,T,F,F,F,
    in_family2 = F,F,F,F,F,F,F,F,F,T,T,T,
    in_family3 = T,F,F,F,F,F,F,F,F,F,F,F,
    in_family4 = T,T,T,T,T,T,T,T,T,T,T,T,
    data_names = "so4", "organic_carbon", "black_carbon", "sea_salt",
                 "dust_0.1", "dust_0.2", "dust_0.4", "dust_0.8",
                 "dust_1.0", "dust_2.0", "dust_4.0", "dust_8.0",
    filename = "aerosol.climatology.nc"
/
 &aerosolrad_package_nml
    volcanic_dataset_entry = 1, 1, 1, 0, 0, 0,
    REPEAT_VOLCANO_YEAR
    VOLCANO_YEAR_USED
    using_volcanic_lw_files = .false.,
    lw_ext_filename = "             "
    lw_ext_root = "     "
    lw_asy_filename = "             "
    lw_asy_root = "     "
    lw_ssa_filename = "             "
    lw_ssa_root = "     "
    using_volcanic_sw_files = .true.,
    sw_ext_filename = "extsw_data.nc"
    sw_ext_root = "extsw"
    sw_ssa_filename = "omgsw_data.nc"
    sw_ssa_root = "omgsw"
    sw_asy_filename = "asmsw_data.nc"
    sw_asy_root = "asmsw"
    do_lwaerosol = .true.,
    do_swaerosol = .true.,
    aerosol_data_set = 'shettle_fenn',
    optical_filename = "aerosol.optical.dat",
    aerosol_optical_names = "sulfate_30%", "sulfate_35%", "sulfate_40%", "sulfate_45%",
                            "sulfate_50%", "sulfate_55%", "sulfate_60%", "sulfate_65%",
                            "sulfate_70%", "sulfate_75%", "sulfate_80%", "sulfate_82%",
                            "sulfate_84%", "sulfate_86%", "sulfate_88%", "sulfate_90%",
                            "sulfate_91%", "sulfate_92%", "sulfate_93%", "sulfate_94%",
                            "sulfate_95%", "sulfate_96%", "sulfate_97%", "sulfate_98%",
                            "sulfate_99%", "sulfate_100%","organic_carbon","soot",
                            "sea_salt",    "dust_0.1",    "dust_0.2",    "dust_0.4",
                            "dust_0.8",    "dust_1.0",    "dust_2.0",    "dust_4.0",
                            "dust_8.0"
.
.
.
&ozone_nml
    BASIC_OZONE_TYPE
    OZONE_DATASET_ENTRY
    data_name = "ozone",
    FILENAME
.
.
.
 &radiation_driver_nml
       rad_time_step= 10800,
       time_varying_solar_constant = .false.,
       solar_dataset_entry = 1990,1,1,0,0,0,
       rad_package = 'sea_esf',
       do_clear_sky_pass=.true.,
       calc_hemi_integrals = .false.,
       renormalize_sw_fluxes=.true.,
       all_step_diagnostics = .true.,
       zenith_spec = 'diurnally_varying'
       using_restart_file = .false.
/
 &radiative_gases_nml
       verbose = 3
       gas_printout_freq = 240
       time_varying_co2 = .true.,
       co2_variation_type = 'linear',
       co2_specification_type = 'time_series',
       co2_floor = 100.0E-06,
       co2_ceiling = 1600.0E-06,
       co2_data_source = 'input'
       time_varying_ch4 = .true.,
       ch4_variation_type = 'linear'
       ch4_specification_type = 'time_series'
       ch4_data_source = 'input'
       time_varying_n2o = .true.,
       n2o_variation_type = 'linear'
       n2o_specification_type = 'time_series'
       n2o_data_source = 'input'
       time_varying_f11 = .true.,
       f11_variation_type = 'linear'
       f11_specification_type = 'time_series'
       f11_data_source = 'input'
       time_varying_f12 = .true.,
       f12_variation_type = 'linear'
       f12_specification_type = 'time_series'
       f12_data_source = 'input'
       time_varying_f113 = .true.,
       f113_variation_type = 'linear'
       f113_specification_type = 'time_series'
       f113_data_source = 'input'
       time_varying_f22 = .true.,
       f22_variation_type = 'linear'
       f22_specification_type = 'time_series'
       f22_data_source = 'input'
```

where the following replacements are made before submission (see https://github.com/csiro-dcfp/cm-forecasts/blob/master/src/run_CAFE_forecasts.sh.in):

 - Replace
   ```
       INPUT_AEROSOL_TIME -> "AYEAR, 1, 1, 0, 0, 0"
   ```
   where AYEAR is the middle year of the decade in which the forecast is initialised (e.g. AYEAR=2005 for any
   start date in the 2000s). If AYEAR is greater than 2015, then 2015 is used.
   
 - If the forecast is initialised after 2000 then replace,
   ```
       REPEAT_VOLCANO_YEAR -> "repeat_volcano_year=.true."
       VOLCANO_YEAR_USED -> "volcano_year_used = 2008,"
   ```
   otherwise these lines are commented out.
   
- If the forecast is initialised after 2004 then replace,
   ```
       BASIC_OZONE_TYPE -> "fixed_year"
       OZONE_DATASET_ENTRY -> "ozone_dataset_entry=2014, 1, 1, 0, 0, 0,"
       FILENAME -> "cm3_2014_o3.padded.nc"
   ```
   otherwise replace,
   ```
       BASIC_OZONE_TYPE -> "time_varying"
       FILENAME -> "CM3_CMIP6_1950-2014_O3.nc"
   ```
   and OZONE_DATASET_ENTRY is commented out.
   
In addition, the following is added to the field table (see https://github.com/csiro-dcfp/cm-forecasts/blob/master/ref/MODEL/field_table_bgc):

- If the forecast is initialised after 2007 then,
   ```
       aco2_file = "INPUT/co2_obs.padded.nc"
   ```
   otherwise,
   ```
       aco2_file = "INPUT/co2_obs.nc"
   ```

I've been told these files are used by the bgc component of the model. Notably, `co2_obs.padded.nc` includes projected values into the future, but both files are identical in their overlapping period so the switch from co2_obs.nc to co2_obs.padded.nc seems to be redundant.

### So what does this mean?

### Aerosols climatologies

Aerosol forcing climatologies are provided per decade. Forecasts initialised prior to 2020 are forced using the climatologies corresponding to the decade of their initialialisation (e.g. 2000s forcing climatologies for every forecast initialised in 2000-2009). Forecasts initialised in 2020 of later use the 2010s aerosol forcing climatologies.

**However**, the aerosol climatologies provided are actually identical to the 2000s climatology for all decades after the 2000s.

Therefore

> <span style="color:red">If initialised during or before 1999, we use the aerosol forcing climatologies corresponding to the decade of initialisation \
If initialised after 1999, we use the aerosol forcing climatologies for the 2000s </span>

### Volcanic forcing
Time varying volcanic forcing is used for forecasts initialised prior to or during 2000, and 2008 volcanic forcing is used for those initialised after 2000.

**However**, the time-varying volcanic forcing provided is actually identical to 1999-12 for all values after 1999-12

Therefore

> <span style="color:red"> We use time varying volcanic forcing up until 1999-12, and then 1999-12 levels from 2000-01 onwards. (Because the forcing files don't actually change after 1999-12, the switch to 2008 forcing in our forecast scripts for initialisation dates after 2000 actually does nothing, other than to allow forecasts to be run beyond the maximum time of the forcing files, 2010-12).<span>

### Ozone forcing
Time varying ozone forcing is used for forecasts initialised prior to or during 2004, and 2014 ozone forcing is used for those initialised after 2004.

Therefore

> <span style="color:red">If initialised during or before 2004, we use time varying monthly (CMIP6) ozone forcing \
If initialised after 2004, we use the 2014 ozone forcing <span>

### Solar forcing
The namelist (see https://github.com/csiro-dcfp/cm-forecasts/blob/master/ref/MODEL/input.in) suggest that fixed 1990 solar forcing is applied for all forecasts.

### Radiative gas forcing
These are taken from the `*_gblannualdata` files (LINK) and are time varying regardless of forecast initialisation date.

Therefore

> <span style="color:red">We use time varying observed and projected radiative gas forcing out to 2110. However, it's not clear to me what scenario is used, or when the switch is made from observations to projections. It looks like `co2_gblannualdata` uses RCP4.5 and `co2_obs.padded.nc` uses RCP 8.5.</span>

## Change in CAFE60v1 bias correction scheme in 1992

The bias correction scheme used for CAFE60v1 (which provides the CAFE-f6 initial conditions) was changed at the beginning of 1992 (or thereabouts). From `O’Kane, T. J., Sandery, P. A., Kitsios, V., Sakov, P., Chamberlain, M. A., Collier, M. A., Fiedler, R., Moore, T. S., Chapman, C. C., Sloyan, B. M., & Matear, R. J. (2021). CAFE60v1: A 60-Year Large Ensemble Climate Reanalysis. Part I: System Design, Model Configuration, and Data Assimilation, Journal of Climate, 34(13), 5153-5169`:

"Prior to 1992, sea surface temperature (SST) is bias corrected using the method of Evensen (2003, section 4.2.2 therein) (see also O’Kane et al. 2019; Sandery et al. 2020). To estimate the SST bias field, an ensemble of bias fields is initialized to independently identically distributed random spatially uniform values and the observed SST is then assumed to be the sum of the model SST and unknown bias. This field is then updated similar to other model fields and the resulting ensemble of biases is evolved using a first-order autoregressive function [see paper for details]. Further, the SST bias influences subsurface temperature in the mixed layer, and therefore affects innovations from subsurface observations. Bias correction in this form can only detect and correct systematic errors in the initial conditions. Due to the relatively long assimilation window (calendar month), model SST bias was found to grow appreciably with lead time in particular regions and very specifically the South Pacific Ocean during the austral summer. Application of weak (90-day time scale) restoring to an SST climatology constructed from HadISST was applied to the forward model post 1992 to ameliorate the biases replacing the method of Evensen (2003)."

This change has a noticable impact on the CAFE-f6 forecasts, with forecasts initialised prior to 1992 demonstrating different behaviour than those initialised after 1992.

## Changes to MOM executable